In [ ]:
#Selenium helps you use this executable to automate Chrome
from multiprocessing.sharedctypes import Value
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from datetime import datetime as dt
from PIL import Image
import time
import os


PATH = '___'  #insica o caminho do webdriver
wd = webdriver.Chrome(executable_path=PATH)


def get_images_from_google(wd, delay, max_images, url):
	def scroll_down(wd):
		wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(delay)

	url = url
	wd.get(url)

	image_urls = set()
	skips = 0
	while len(image_urls) + skips < max_images:
		scroll_down(wd)
		thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

		for img in thumbnails[len(image_urls) + skips:max_images]:
			try:
				img.click()
				time.sleep(delay)
			except:
				continue

			images = wd.find_elements(By.CLASS_NAME, "n3VNCb")
			for image in images:
				if image.get_attribute('src') in image_urls:
					max_images += 1
					skips += 1
					break

				if image.get_attribute('src') and 'http' in image.get_attribute('src'):
					image_urls.add(image.get_attribute('src'))
					

	return image_urls


def download_image(down_path, url, file_name, image_type='JPEG',
                   verbose=True):
    try:
        time = dt.now()
        curr_time = time.strftime('%H:%M:%S')
        #Content of the image will be a url
        img_content = requests.get(url).content
        #Get the bytes IO of the image
        img_file = io.BytesIO(img_content)
        #Stores the file in memory and convert to image file using Pillow
        image = Image.open(img_file)
        file_pth = down_path + file_name

        with open(file_pth, 'wb') as file:
            image.save(file, image_type)

        if verbose == True:
            print(f'The image: {file_pth} downloaded successfully at {curr_time}.')
    except Exception as e:
        print(f'Unable to download image from Google Photos due to\n: {str(e)}')



if __name__ == '__main__':
    
    google_urls = ['https://www.google.com/search?q=dogs&tbm=isch&ved=2ahUKEwiSqb3c-dD5AhV1MrkGHdN3DRYQ2-cCegQIABAA&oq=dogs&gs_lcp=CgNpbWcQAzIICAAQgAQQsQMyBwgAELEDEEMyCAgAEIAEELEDMggIABCABBCxAzIHCAAQsQMQQzIICAAQgAQQsQMyBAgAEEMyBAgAEEMyBAgAEEMyCAgAEIAEELEDOgQIIxAnOgUIABCABDoLCAAQgAQQsQMQgwE6CAgAELEDEIMBUJEFWP0IYOUJaABwAHgAgAG_AYgBqgWSAQMwLjSYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=vHv-YpLbF_Xk5OUP0--1sAE&bih=656&biw=1536&rlz=1C1GCEA_enBR1007BR1007',
                   'https://www.google.com/search?q=cats&tbm=isch&ved=2ahUKEwi805re-dD5AhXnMrkGHfxaChUQ2-cCegQIABAA&oq=cats&gs_lcp=CgNpbWcQAzIHCAAQsQMQQzIECAAQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIECAAQQzIFCAAQgAQ6BAgjECc6CAgAELEDEIMBOgoIABCxAxCDARBDOgsIABCABBCxAxCDAToICAAQgAQQsQNQpgVY5ghg3gpoAHAAeACAAb8BiAHfBpIBAzAuNZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=wHv-Yrxz5-Xk5Q_8tamoAQ&bih=656&biw=1536&rlz=1C1GCEA_enBR1007BR1007',
                   'https://www.google.com/search?q=lions+animals&tbm=isch&ved=2ahUKEwji1oP6-dD5AhXRALkGHQZDCUcQ2-cCegQIABAA&oq=lions+animals&gs_lcp=CgNpbWcQAzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzoECCMQJzoFCAAQgAQ6BAgAEEM6BAgAEB5QpQNYsApgoAtoAHAAeACAAcYBiAGZC5IBAzAuOJgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=-nv-YuLxFdGB5OUPhoaluAQ&bih=656&biw=1536&rlz=1C1GCEA_enBR1007BR1007'
    ]
    
    # Inserindo as labels que irão para as pastas
    labels = [
        'Cachorros', 'Gatos', 'Leões'
    ]

    # Verificando tamanho da lista
    if len(google_urls) != len(labels):
        raise ValueError('The length of the url list does not match the labels list.')

    player_path = '___' #path para download
     # player_path = 'insira a pasta que vai armazenar as imagens'
   
    for lbl in labels:
        if not os.path.exists(player_path + lbl):
            print(f'Making directory: {str(lbl)}')
            os.makedirs(player_path+lbl)

    for url_current, lbl in zip(google_urls, labels):
        urls = get_images_from_google(wd, 0, 200, url_current)

        # Once we have added our urls to empty set then 
        for i, url in enumerate(urls):
            download_image(down_path=f'___\\{lbl}\\',  #path para download
                        url=url,
                        file_name=f"{str(lbl)}{(str(i+1))}.jpg",
                        #file_name=str(i+1)+ '.jpg',
                        verbose=True) 
    wd.quit()

